In [3]:
library(dplyr)
library(reshape2)
library(plotly)
library(lubridate)


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union




In [4]:
energy_data<-read.csv("energydata_complete.csv")
energy_data$date <- strptime(as.character(energy_data$date),format="%Y-%m-%d %H:%M:%S") 
energy_data$date <- as.POSIXct(energy_data$date,tz = "UTC") 
energy_data$mhr <- floor_date(energy_data$date,"hour")
energy_data$Day_of_week <- as.factor(energy_data$Day_of_week)
energy_data$WeekStatus <- unlist(lapply(energy_data$date, weekend_weekday))

ERROR: Error in `$<-.data.frame`(`*tmp*`, Day_of_week, value = structure(integer(0), .Label = character(0), class = "factor")): replacement has 0 rows, data has 19735


#### Analyzing the energy trend of appliances with respect to the date & time 

## Detect the impact of temperature outside on the consumed energy on appliances
Detect the impact of temperature outside for the first week of January 2016 on the appliances energy consumed in order to maintain the temperature inside the living room of a house. The plots plotted are below:

Left plot - Weekday v/s weekend energy consumed for January
Right plot - Impact of outside temperature on the temperature of living room

Comparison has been made for the weekdays and weekends consumed units of energy in Kwh during the winter season.

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
temperature_subset <- select(energy_data, date, T2, T_out)
temperature_subset_melt <- melt(temperature_subset, id = c("date"))
temperature_melt_subset <- subset(temperature_subset_melt, date > "2016-01-10" & date < "2016-01-17")
plot2 <- ggplot(temperature_melt_subset, aes(x = date, y = value, colour = variable)) +
  geom_line() +
  labs(title = "TEMPERATURE INSIDE v/s OUTSIDE FOR JANUARY") +
  theme_bw() +
  theme(text = element_text(size = 11, family = "serif")) +
  labs(y = "TEMPERATURE (in Celius) ", x = "JANUARY 1st WEEK", col = "SIDE")
#plot2 <- ggplotly(plot2, width = 700, height = 350)
plot2

In [5]:
energy_data$Day_of_week <- weekdays(energy_data$date)
weekend_weekday <- function(x) {
  val <- weekdays(x)
  if (val == "Saturday" | val == "Sunday") {
    val2 <- "Weekend"
  } else {
    val2 <- "Weekday"
  }
  return(val2)
}

In [ ]:
energy_data$Day_of_week <- as.factor(energy_data$Day_of_week)
energy_data$WeekStatus <- unlist(lapply(energy_data$date, weekend_weekday))
data_weekstatus <- select(energy_data, date, WeekStatus, Appliances, mhr)
data_weekstatus$Appliances <- data_weekstatus$Appliances/1000
data_weekstatus_subset <- subset(data_weekstatus, date > "2016-01-10" & date < "2016-01-17")
options(repr.plot.width=8, repr.plot.height=6)
plot1 <- ggplot(data_weekstatus_subset, aes(x = date, y = Appliances, colour = WeekStatus)) +
  geom_line() +
  ylim(0, 1) +
  labs(title = "WEEKDAY v/s WEEKEND ENERGY CONSUMED FOR JANUARY") +
  theme_bw() +
  theme(text = element_text(size = 11, family = "serif")) +
  labs(y = "APPLIANCE - energy use in KWh", x = "JANUARY 1st WEEK", col = "WEEKSTATUS")

In [ ]:
options(repr.plot.width=20, repr.plot.height=8)
plot1 + plot2 + plot_annotation(
  title = 'Impact of temperature outside on the consumed energy',theme = theme(plot.title = element_text(size = 25,face="bold")))